### Below we write a program to accomplish two things:
#### (1) We calculate the point cloud corresponding to each pair of RGB-D images based on internal parameters;
#### (2) According to the camera pose of each image, we put the points to a global cloud by the camera poses.

#### Import required packages

In [64]:
import numpy as np
import cv2
import open3d as o3d
import sys
sys.path.append('../../../')
from src.software.utils.osaQuaternion import simpleQuaternion

#### Read Image data from RGB and Depth Folder

In [13]:
rgb_images = []
depth_images = []
for i in range(5):
    rgb = cv2.imread("color/" + str(i+1) + ".png")
    depth = cv2.imread("depth/" + str(i+1) + ".pgm", -1)
    rgb_images.append(rgb)
    depth_images.append(depth)

#### Read trajectory file that consists of rotation and translation

In [28]:
trajectory = np.loadtxt("pose.txt", delimiter=" ", dtype=float)
translations = trajectory[:, :3]
rotations = np.roll(trajectory[:, 3:], 1, axis=1)

#### Compute the point clouds using camera intrinsics


In [106]:
# Intrensic and default depthScale
cx = 325.5
cy = 253.5
fx = 518.0
fy = 519.0
depthScale = 1000.0
rows, cols, _ = rgb_images[0].shape

pointclouds = np.zeros(6)
for i in range(5):
    points = []
    colors = []
    for v in range(rows):
        for u in range(cols):
            if depth_images[i][v][u] == 0:
                continue
            z_val = depth_images[i][v][u] / depthScale
            x_val = (u - cx) * z_val / fx
            y_val = (v - cx) * z_val / fy
            points.append([x_val, y_val, z_val])
            colors.append(rgb_images[i][v][u]/255)
    rotator = simpleQuaternion(rotations[i])
    points = rotator.rotate(np.asarray(points))
    points = points + translations[i]
    colors = np.asarray(colors)
    colors[:, [2, 0]] = colors[:, [0, 2]]
    pointclouds = np.vstack([pointclouds, np.hstack([points, colors])])

TypeError: list indices must be integers or slices, not tuple

#### Visualise image point cloud

In [102]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointclouds[:, :3])
pcd.colors = o3d.utility.Vector3dVector(pointclouds[:, 3:])

o3d.visualization.draw_geometries([pcd])